In [ ]:
fold = 0

import torch
torch.manual_seed(42)
import random
random.seed(42)

In [ ]:
import pandas as pd

sentence_only_with_folds_df = pd.read_csv("ANON")
fine_tune_df = pd.read_csv(f"ANON")
few_shot_of_wrong_guesses_of_fine_tune_df = pd.read_csv(f"ANON")
few_shot_of_wrong_guesses_of_fine_tune_df.head()

In [ ]:
fine_tune_df.head()

In [ ]:
filter_and_relabel_outputs = pd.read_csv(f"ANON")
filter_and_relabel_outputs.head()

In [ ]:
# rows in filter_and_relabel_outputs that new_label and original_label are different
diff_df = filter_and_relabel_outputs[filter_and_relabel_outputs["new_label"] != filter_and_relabel_outputs["original_label"]]
print(len(fine_tune_df))
print(len(diff_df))
fine_tune_df = fine_tune_df[~fine_tune_df.text.isin(diff_df.text)]
print(len(fine_tune_df))
diff_df.head()

In [ ]:
filter_and_relabel_outputs_wrong_preds = pd.read_csv(f"ANON")
filter_and_relabel_outputs_wrong_preds.head()



In [ ]:
# rows in filter_and_relabel_outputs_wrong_preds that new_label and original_label are different
diff_df_wrong_preds = filter_and_relabel_outputs_wrong_preds[filter_and_relabel_outputs_wrong_preds["new_label"] != filter_and_relabel_outputs_wrong_preds["original_label"]]
print(len(few_shot_of_wrong_guesses_of_fine_tune_df))
print(len(diff_df_wrong_preds))
few_shot_of_wrong_guesses_of_fine_tune_df = few_shot_of_wrong_guesses_of_fine_tune_df[~few_shot_of_wrong_guesses_of_fine_tune_df.text.isin(diff_df_wrong_preds.text)]
print(len(few_shot_of_wrong_guesses_of_fine_tune_df))
diff_df_wrong_preds.head()

In [ ]:
print(len(sentence_only_with_folds_df))
train_df = sentence_only_with_folds_df[sentence_only_with_folds_df.test_fold != fold]
# remove columns text-id, id, test_fold
train_df = train_df.drop(columns=['id', 'test_fold'])
print(len(train_df))

print(len(fine_tune_df))
train_df_2 = fine_tune_df[fine_tune_df.test_fold != fold]
# remove columns text-id, id, test_fold, original_id, test_fold
train_df_2 = train_df_2.drop(columns=['id', 'test_fold', 'original_id', 'test_fold'])
print(len(train_df_2))

train_df_3 = few_shot_of_wrong_guesses_of_fine_tune_df.drop(columns=['id', 'original_id'])

train_df = pd.concat([train_df, train_df_2, train_df_3])

print(len(train_df))
train_df.head()

In [ ]:
# shuffle train_df with seed 42
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
train_df.head()

In [ ]:
from simpletransformers.classification import ClassificationModel
model = ClassificationModel('bert', 'bert-base-german-cased', num_labels=max(train_df.label) + 1, use_cuda=False, args={'output_dir': f'output-fold-{fold}', 'num_train_epochs': 3})
model.train_model(train_df)

In [ ]:
test_df = sentence_only_with_folds_df[sentence_only_with_folds_df.test_fold == fold]

# find F1 score per label
from sklearn.metrics import f1_score, balanced_accuracy_score
import numpy as np

def f1_score_per_label(y_true, y_pred):
    return np.array([f1_score(y_true == i, y_pred == i) for i in range(max(y_true) + 1)])

predictions, raw_outputs = model.predict(test_df.text.to_list())
list_of_f1 = list(f1_score_per_label(test_df.label, predictions))

overall_f1 = f1_score(test_df.label, predictions, average='weighted')
overall_balanced_accuracy = balanced_accuracy_score(test_df.label, predictions)
with open(f'f1-fold-{fold}.txt', 'w') as f:
    for item in list_of_f1 + [overall_f1, overall_balanced_accuracy]:
        f.write("%s\n" % item)

In [ ]:
# find the texts that the model got wrong, print 5 of them with the correct and predicted labels
predictions_train, raw_outputs_train = model.predict(train_df.text.to_list())
wrong_df = train_df[train_df.label != predictions_train]
wrong_df.to_csv(f'wrong-predictions-fold-{fold}.csv', index=False)